In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings('ignore')

%matplotlib inline

We've talked about Random Forests. Now it's time to build one.

Here we'll use data from Lending Club (2015) to predict the state of a loan given some information about it. You can download the dataset [here](https://www.dropbox.com/s/0so14yudedjmm5m/LoanStats3d.csv?dl=1)

In [2]:
# Replace the path with the correct path for your data.
y2015 = pd.read_csv(
    'https://www.dropbox.com/s/0so14yudedjmm5m/LoanStats3d.csv?dl=1',
    skipinitialspace=True,
    header=1
)

# Note the warning about dtypes.

//anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (0,19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
y2015.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,68009401,72868139.0,16000.0,16000.0,16000.0,60 months,14.85%,379.39,C,C5,...,0.0,2.0,78.9,0.0,0.0,2.0,298100.0,31329.0,281300.0,13400.0
1,68354783,73244544.0,9600.0,9600.0,9600.0,36 months,7.49%,298.58,A,A4,...,0.0,2.0,100.0,66.7,0.0,0.0,88635.0,55387.0,12500.0,75635.0
2,68466916,73356753.0,25000.0,25000.0,25000.0,36 months,7.49%,777.55,A,A4,...,0.0,0.0,100.0,20.0,0.0,0.0,373572.0,68056.0,38400.0,82117.0
3,68466961,73356799.0,28000.0,28000.0,28000.0,36 months,6.49%,858.05,A,A2,...,0.0,0.0,91.7,22.2,0.0,0.0,304003.0,74920.0,41500.0,42503.0
4,68495092,73384866.0,8650.0,8650.0,8650.0,36 months,19.89%,320.99,E,E3,...,0.0,12.0,100.0,50.0,1.0,0.0,38998.0,18926.0,2750.0,18248.0


## The Blind Approach

Now, as we've seen before, creating a model is the easy part. Let's try just using everything we've got and throwing it without much thought into a Random Forest. SKLearn requires the independent variables to be be numeric, and all we want is dummy variables so let's use `get_dummies` from Pandas to generate a dummy variable for every categorical column and see what happens off of this kind of naive approach.

In [3]:
from sklearn import ensemble
from sklearn.model_selection import cross_val_score

rfc = ensemble.RandomForestClassifier()
X = y2015.drop('loan_status', 1)
Y = y2015['loan_status']

Did your kernel die? My kernel died.

Guess it isn't always going to be that easy...

Can you think of what went wrong?

(You're going to have to reset your kernel and reload the column, BUT DON'T RUN THE MODEL AGAIN OR YOU'LL CRASH THE KERNEL AGAIN!)

## Data Cleaning

Well, `get_dummies` can be a very memory intensive thing, particularly if data are typed poorly. We got a warning about that earlier. Mixed data types get converted to objects, and that could create huge problems. Our dataset is about 400,000 rows. If there's a bad type there its going to see 400,000 distinct values and try to create dummies for all of them. That's bad. Lets look at all our categorical variables and see how many distinct counts there are...

In [4]:
categorical = y2015.select_dtypes(include=['object'])
for i in categorical:
    column = categorical[i]
    print(i)
    print(column.nunique())

id
421097
term
2
int_rate
110
grade
7
sub_grade
35
emp_title
120812
emp_length
12
home_ownership
4
verification_status
3
issue_d
12
loan_status
7
pymnt_plan
1
url
421095
desc
34
purpose
14
title
27
zip_code
914
addr_state
49
earliest_cr_line
668
revol_util
1211
initial_list_status
2
last_pymnt_d
25
next_pymnt_d
4
last_credit_pull_d
26
application_type
2
verification_status_joint
3


Well that right there is what's called a problem. Some of these have over a hundred thousand distinct types. Lets drop the ones with over 30 unique values, converting to numeric where it makes sense. In doing this there's a lot of code that gets written to just see if the numeric conversion makes sense. It's a manual process that we'll abstract away and just include the conversion.

You could extract numeric features from the dates, but here we'll just drop them. There's a lot of data, it shouldn't be a huge problem.

In [4]:
# Convert ID and Interest Rate to numeric.
y2015['id'] = pd.to_numeric(y2015['id'], errors='coerce')
y2015['int_rate'] = pd.to_numeric(y2015['int_rate'].str.strip('%'), errors='coerce')

# Drop other columns with many unique variables
y2015.drop(['url', 'emp_title', 'zip_code', 'earliest_cr_line', 'revol_util',
            'sub_grade', 'addr_state', 'desc'], 1, inplace=True)

Wonder what was causing the dtype error on the id column, which _should_ have all been integers? Let's look at the end of the file.

In [6]:
y2015.tail()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
421092,36271333.0,38982739.0,13000.0,13000.0,13000.0,60 months,15.99,316.07,D,5 years,...,0.0,3.0,100.0,50.0,1.0,0.0,51239.0,34178.0,10600.0,33239.0
421093,36490806.0,39222577.0,12000.0,12000.0,12000.0,60 months,19.99,317.86,E,1 year,...,1.0,2.0,95.0,66.7,0.0,0.0,96919.0,58418.0,9700.0,69919.0
421094,36271262.0,38982659.0,20000.0,20000.0,20000.0,36 months,11.99,664.20,B,10+ years,...,0.0,1.0,100.0,50.0,0.0,1.0,43740.0,33307.0,41700.0,0.0
421095,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
421096,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Remove two summary rows at the end that don't actually contain data.
y2015 = y2015[:-2]

Now this should be better. Let's try again.

In [6]:
pd.get_dummies(y2015)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,...,last_credit_pull_d_Nov-2016,last_credit_pull_d_Oct-2015,last_credit_pull_d_Oct-2016,last_credit_pull_d_Sep-2015,last_credit_pull_d_Sep-2016,application_type_INDIVIDUAL,application_type_JOINT,verification_status_joint_Not Verified,verification_status_joint_Source Verified,verification_status_joint_Verified
0,68009401.0,72868139.0,16000.0,16000.0,16000.0,14.85,379.39,48000.0,33.18,0.0,...,0,0,0,0,0,1,0,0,0,0
1,68354783.0,73244544.0,9600.0,9600.0,9600.0,7.49,298.58,60000.0,22.44,0.0,...,0,0,0,0,0,1,0,0,0,0
2,68466916.0,73356753.0,25000.0,25000.0,25000.0,7.49,777.55,109000.0,26.02,0.0,...,0,0,0,0,0,1,0,0,0,0
3,68466961.0,73356799.0,28000.0,28000.0,28000.0,6.49,858.05,92000.0,21.60,0.0,...,0,0,0,0,0,1,0,0,0,0
4,68495092.0,73384866.0,8650.0,8650.0,8650.0,19.89,320.99,55000.0,25.49,0.0,...,0,0,0,0,0,1,0,0,0,0
5,68506798.0,73396623.0,23000.0,23000.0,23000.0,8.49,471.77,64000.0,18.28,0.0,...,0,0,0,0,0,1,0,0,0,0
6,68566886.0,73456723.0,29900.0,29900.0,29900.0,12.88,678.49,65000.0,21.77,0.0,...,0,0,0,0,0,1,0,0,0,0
7,68577849.0,73467703.0,18000.0,18000.0,18000.0,11.99,400.31,112000.0,8.68,0.0,...,0,0,0,0,0,1,0,0,0,0
8,66310712.0,71035433.0,35000.0,35000.0,35000.0,14.85,829.90,110000.0,17.06,0.0,...,0,0,0,0,0,1,0,0,0,0
9,68476807.0,73366655.0,10400.0,10400.0,10400.0,22.45,289.91,104433.0,25.37,1.0,...,0,0,0,0,0,1,0,0,0,0


It finally works! We had to sacrifice sub grade, state address and description, but that's fine. If you want to include them you could run the dummies independently and then append them back to the dataframe.

## Second Attempt

Now let's try this model again.

We're also going to drop NA columns, rather than impute, because our data is rich enough that we can probably get away with it.

This model may take a few minutes to run.

In [7]:
from sklearn import ensemble
from sklearn.model_selection import cross_val_score

rfc = ensemble.RandomForestClassifier()
X = y2015.drop('loan_status', 1)
Y = y2015['loan_status']
X = pd.get_dummies(X)
X = X.dropna(axis=1)

cross_val_score(rfc, X, Y, cv=10)

//anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
//anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
//anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
//anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
//anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: T

array([0.97917409, 0.98021895, 0.98140628, 0.98164375, 0.96962717,
       0.97967229, 0.95084186, 0.98052673, 0.97988458, 0.97990785])

The score cross validation reports is the accuracy of the tree. Here we're about 98% accurate.

That works pretty well, but there are a few potential problems. Firstly, we didn't really do much in the way of feature selection or model refinement. As such there are a lot of features in there that we don't really need. Some of them are actually quite impressively useless.

There's also some variance in the scores. The fact that one gave us only 93% accuracy while others gave higher than 98 is concerning. This variance could be corrected by increasing the number of estimators. That will make it take even longer to run, however, and it is already quite slow.

## DRILL: Third Attempt

So here's your task. Get rid of as much data as possible without dropping below an average of 90% accuracy in a 10-fold cross validation.

You'll want to do a few things in this process. First, dive into the data that we have and see which features are most important. This can be the raw features or the generated dummies. You may want to use PCA or correlation matrices.

Can you do it without using anything related to payment amount or outstanding principal? How do you know?

In [59]:
import scipy.stats as ss

# Importing code for Cramers V, a measure similar to correlation 
# coefficient for categorical variables
def cramers_v(x, y):
    confusion_matrix = pd.crosstab(x,y)
    chi2 = ss.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2/n
    r,k = confusion_matrix.shape
    phi2corr = max(0, phi2-((k-1)*(r-1))/(n-1))
    rcorr = r-((r-1)**2)/(n-1)
    kcorr = k-((k-1)**2)/(n-1)
    return np.sqrt(phi2corr/min((kcorr-1),(rcorr-1)))


In [90]:
X = X.drop(['id', 'member_id'], 1)

Let's look at which variables are most correlated with our target.

In [34]:
cramers = pd.DataFrame(X.columns)
cramers = cramers.rename(columns={0: 'column'})

In [46]:
# This code takes several minutes
cramers['cv'] = cramers.apply(lambda x: cramers_v(Y,X[x['column']]), axis='columns')

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in double_scalars
  


In [55]:
cramers.sort_values(by=['cv'], ascending=False)

,column,cv
22,last_pymnt_amnt,0.178012
160,last_pymnt_d_Oct-2015,0.172469
158,last_pymnt_d_Nov-2015,0.166003
169,last_credit_pull_d_Apr-2016,0.162715
183,last_credit_pull_d_Jun-2016,0.154126
185,last_credit_pull_d_Mar-2016,0.153928
162,last_pymnt_d_Sep-2015,0.150649
19,total_rec_late_fee,0.148290
187,last_credit_pull_d_May-2016,0.145296
193,last_credit_pull_d_Sep-2016,0.144597


The model is using variables like "last_pymnt_d" to "predict," whether a loan will be paid back. This variable refers to the last month a payment was received. While this would obviously be correlated with whether a loan was paid back, we wouldn't have this information when we're investing, so it seems cheap to use it in our model.

In [54]:
for i in range(y2015.shape[0]):
    print('{}: {}'.format(i, y2015.columns[i]))

0: id
1: member_id
2: loan_amnt
3: funded_amnt
4: funded_amnt_inv
5: term
6: int_rate
7: installment
8: grade
9: emp_length
10: home_ownership
11: annual_inc
12: verification_status
13: issue_d
14: loan_status
15: pymnt_plan
16: purpose
17: title
18: dti
19: delinq_2yrs
20: inq_last_6mths
21: mths_since_last_delinq
22: mths_since_last_record
23: open_acc
24: pub_rec
25: revol_bal
26: total_acc
27: initial_list_status
28: out_prncp
29: out_prncp_inv
30: total_pymnt
31: total_pymnt_inv
32: total_rec_prncp
33: total_rec_int
34: total_rec_late_fee
35: recoveries
36: collection_recovery_fee
37: last_pymnt_d
38: last_pymnt_amnt
39: next_pymnt_d
40: last_credit_pull_d
41: collections_12_mths_ex_med
42: mths_since_last_major_derog
43: policy_code
44: application_type
45: annual_inc_joint
46: dti_joint
47: verification_status_joint
48: acc_now_delinq
49: tot_coll_amt
50: tot_cur_bal
51: open_acc_6m
52: open_il_6m
53: open_il_12m
54: open_il_24m
55: mths_since_rcnt_il
56: total_bal_il
57: il_uti

IndexError: index 103 is out of bounds for axis 0 with size 103

In [56]:
y2015.columns[28:41]

Index(['out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv',
       'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries',
       'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt',
       'next_pymnt_d', 'last_credit_pull_d'],
      dtype='object')

In [57]:
y2015 = y2015.drop(y2015.columns[28:41], 1)

In [58]:
rfc = ensemble.RandomForestClassifier()
X = y2015.drop('loan_status', 1)
Y = y2015['loan_status']
X = pd.get_dummies(X)
X = X.dropna(axis=1)

cross_val_score(rfc, X, Y, cv=10)

//anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
//anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
//anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
//anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
//anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: T

array([0.68250576, 0.29510104, 0.26290043, 0.26959702, 0.17900736,
       0.11398718, 0.11078392, 0.0834026 , 0.15021255, 0.24459697])

Our model performs dramatically worse once we remove these future variables. For the sake of the assignment, I'm going to add them back in. However, if we are actually concerned about the model's predictive power, it's performance as-is looks poor.

In [60]:
y2015 = pd.read_csv(
    'https://www.dropbox.com/s/0so14yudedjmm5m/LoanStats3d.csv?dl=1',
    skipinitialspace=True,
    header=1
)

//anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (0,19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [61]:
y2015['id'] = pd.to_numeric(y2015['id'], errors='coerce')
y2015['int_rate'] = pd.to_numeric(y2015['int_rate'].str.strip('%'), errors='coerce')

# Drop other columns with many unique variables
y2015.drop(['url', 'emp_title', 'zip_code', 'earliest_cr_line', 'revol_util',
            'sub_grade', 'addr_state', 'desc'], 1, inplace=True)

y2015 = y2015[:-2]
pd.get_dummies(y2015)

X = y2015.drop('loan_status', 1)
Y = y2015['loan_status']
X = pd.get_dummies(X)
X = X.dropna(axis=1)

In [65]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

sklearn_pca = PCA(n_components=5)
X_temp = sklearn_pca.fit_transform(X)

rfc = ensemble.RandomForestClassifier()
cross_val_score(rfc, X_temp, Y, cv=10)

//anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
//anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
//anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
//anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
//anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: T

array([0.68157963, 0.21847023, 0.16993185, 0.16458882, 0.11201615,
       0.08888625, 0.07157615, 0.07589826, 0.13249578, 0.24823066])

In [70]:
# Warning: takes over an hour to run.

for i in range(1,20):
    sklearn_pca = PCA(n_components=5*i)
    X_temp = sklearn_pca.fit_transform(X)

    rfc = ensemble.RandomForestClassifier()
    scores = cross_val_score(rfc, X_temp, Y, cv=10)
    
    print('-------------------------- {} components --------------------------'.format(str(i*5)))
    print('')
    print(scores)
    print('')
    print('')
          
    if min(scores) > .9:
        break

-------------------------- 5 components --------------------------

[0.68198333 0.21749662 0.16997934 0.1621904  0.11332225 0.09071479
 0.06905887 0.07392719 0.12990714 0.24659193]


-------------------------- 10 components --------------------------

[0.69905725 0.76305478 0.766973   0.75992021 0.64438376 0.63488483
 0.52786815 0.53496877 0.52017479 0.58032109]


-------------------------- 15 components --------------------------

[0.79492294 0.94398138 0.94792335 0.95112916 0.95160294 0.94958442
 0.94806336 0.94946449 0.94551975 0.93435615]


-------------------------- 20 components --------------------------

[0.82619743 0.9493244  0.94965686 0.9537413  0.95713607 0.95115175
 0.95542521 0.9543328  0.9550906  0.93789484]


-------------------------- 25 components --------------------------

[0.80867232 0.95663841 0.96141151 0.96046164 0.96162432 0.96138684
 0.95865492 0.95860742 0.95594557 0.9402223 ]


-------------------------- 30 components --------------------------

[0.82793094 

65 Components is the least needed to achieve the desired level of accuracy using PCA.

In [89]:
topTen = cramers.sort_values(by=['cv'], ascending=False)['column'][:10]

XX = X[topTen]

rfc = ensemble.RandomForestClassifier()
cross_val_score(rfc, XX, Y, cv=10)

array([0.93721355, 0.93220299, 0.93752226, 0.94507373, 0.90916647,
       0.93873189, 0.93607067, 0.94592605, 0.9248581 , 0.95573077])

In [88]:
topNine = cramers.sort_values(by=['cv'], ascending=False)['column'][:9]

XX = X[topNine]

rfc = ensemble.RandomForestClassifier()
cross_val_score(rfc, XX, Y, cv=10)

array([0.93733229, 0.93156182, 0.93633492, 0.94367267, 0.9053194 ,
       0.93968179, 0.93645064, 0.94566482, 0.92675802, 0.95641951])

In [82]:
topEight = cramers.sort_values(by=['cv'], ascending=False)['column'][:8]

XX = X[topEight]

rfc = ensemble.RandomForestClassifier()
cross_val_score(rfc, XX, Y, cv=10)

array([0.93930327, 0.93633492, 0.94087056, 0.94675975, 0.91524578,
       0.9417478 , 0.93685435, 0.94787338, 0.92891918, 0.95651451])

In [84]:
topSeven = cramers.sort_values(by=['cv'], ascending=False)['column'][:7]

XX = X[topSeven]

rfc = ensemble.RandomForestClassifier()
cross_val_score(rfc, XX, Y, cv=10)

array([0.94357769, 0.9317518 , 0.93588374, 0.94747216, 0.91548326,
       0.94041795, 0.93414709, 0.94357501, 0.9200133 , 0.95238208])

In [85]:
topSix = cramers.sort_values(by=['cv'], ascending=False)['column'][:6]

XX = X[topSix]

rfc = ensemble.RandomForestClassifier()
cross_val_score(rfc, XX, Y, cv=10)

array([0.94362518, 0.93483888, 0.93642991, 0.94846952, 0.91652814,
       0.94077416, 0.93673561, 0.94542734, 0.921177  , 0.95356956])

In [86]:
topFive = cramers.sort_values(by=['cv'], ascending=False)['column'][:5]

XX = X[topFive]

rfc = ensemble.RandomForestClassifier()
cross_val_score(rfc, XX, Y, cv=10)

array([0.94338771, 0.93267792, 0.93346157, 0.94659353, 0.9146996 ,
       0.94006174, 0.93528699, 0.94336128, 0.91946707, 0.95283333])

In [87]:
topFour = cramers.sort_values(by=['cv'], ascending=False)['column'][:4]

XX = X[topFour]

rfc = ensemble.RandomForestClassifier()
cross_val_score(rfc, XX, Y, cv=10)

array([0.93165681, 0.91054594, 0.9031132 , 0.93217924, 0.88919497,
       0.91726431, 0.91301147, 0.91365266, 0.89517182, 0.92217261])

It looks like, by taking the 5 most correlated features, we can get the same accuracy as using 65 components. This is an interesting result, and I wonder if it would hold if the features were less correlated with our target (my guess would be no).